In [13]:
%pip install esgf-pyclient==0.3.1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
     -------------------------------------- 134.9/134.9 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 64.8/64.8 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 61.4/61.4 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 115.4/115.4 kB 6.6 MB/s eta 0:00:00
     ---------------------------------------- 45.4/45.4 kB ? eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 105.4/105.4 kB ? eta 0:00:00
  Using cached idna-3.10-py3-none-any.whl (70 kB)
     -------------------------------------- 129.8/129.8 kB 8.0 MB/s eta 0:00:00
     ------------------------------------- 162.7/162.7 kB 10.2 MB/s eta 0:00:00
  Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
     ---------------------

  DEPRECATION: esgf-pyclient is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### listagem de variáveis do CMPI6

In [1]:
import sys

# apontar para a versão local da lib
sys.path.insert(0, './dreqPy')

In [2]:
from dreqPy import dreq

In [3]:
# inicializando a biblioteca
dq = dreq.loadDreq()

In [4]:
# listando variáveis relacionadas à temperatura
temperature_vars = [var for var in dq.coll["var"].items if "temperature" in var.title.lower()]

for v in temperature_vars:
    print(v.label, "-", v.title, "-", v.sn)

bigthetao - Sea Water Conservative Temperature - sea_water_conservative_temperature
bigthetaoga - Global Average Sea Water Conservative Temperature - sea_water_conservative_temperature
edt - Eddy Diffusivity Coefficient for Temperature - atmosphere_heat_diffusivity
hfevapds - Temperature Flux Due to Evaporation Expressed as Heat Flux out of Sea Water - temperature_flux_due_to_evaporation_expressed_as_heat_flux_out_of_sea_water
hfrainds - Temperature Flux Due to Rainfall Expressed as Heat Flux into Sea Water - temperature_flux_due_to_rainfall_expressed_as_heat_flux_into_sea_water
hfrunoffds - Temperature Flux Due to Runoff Expressed as Heat Flux into Sea Water - temperature_flux_due_to_runoff_expressed_as_heat_flux_into_sea_water
litempbotfl - Basal Temperature of Floating Ice Shelf - land_ice_basal_temperature
litempbotgr - Basal Temperature of Grounded Ice Sheet - land_ice_basal_temperature
litemptop - Temperature at Top of Ice Sheet Model - temperature_at_top_of_ice_sheet_model
litem

### acesso ao ESGF

In [56]:
from collections import Counter, defaultdict
import logging
from pyesgf.search import SearchConnection

In [6]:
logging.getLogger('pyesgf.search').setLevel(logging.ERROR)

In [40]:
# variáveis relevantes de acordo com GCOS
variaveis = ["tas", "tasmax", "tasmin", "ps", "pr", "hur", "rsds", "rlut", "uas", "vas", "tos", "sos", "sic"]

In [55]:
conn = SearchConnection("https://esgf-node.llnl.gov/esg-search", distrib=False)   # distrib = False —> força a busca apenas no LLNL, que costuma estar mais estável

ctx = conn.new_context(
    project="CMIP6",
    experiment_id=["historical", "ssp245", "ssp585"],
    variable_id=variaveis,
    frequency=["mon"],
    #member_id="r1i1p1f1",
    #grid_label="gn",
    replica=False,
    latest=True
)

# armazenando os resultados
results = list(ctx.search(ignore_facet_check=True))

# extraindo os modelos (source_id)
models = [d.json.get("source_id", [None])[0] for d in results if "source_id" in d.json]

# frequência de cada modelo
print(Counter(models))

"""
mapping = {}

for d in ctx.search(ignore_facet_check=True):
    models = [d.json.get("source_id", [None])[0] for d in ctx.search(ignore_facet_check=True)]
    print(Counter(models))

    model = d.json.get("source_id", [None])[0]
    experiment = d.json.get("experiment_id", [None])[0]
    variable = d.json.get("variable_id", [None])[0]
    
    if model and experiment and variable:
        mapping.setdefault(model, {}).setdefault(experiment, set()).add(variable)

# Exibir resultados
for model, exps in mapping.items():
    print(f"\nModelo: {model}")
    for exp, vars in exps.items():
        print(f"  Experimento: {exp} → Variáveis: {sorted(vars)}")"""

Counter()


'\nmapping = {}\n\nfor d in ctx.search(ignore_facet_check=True):\n    models = [d.json.get("source_id", [None])[0] for d in ctx.search(ignore_facet_check=True)]\n    print(Counter(models))\n\n    model = d.json.get("source_id", [None])[0]\n    experiment = d.json.get("experiment_id", [None])[0]\n    variable = d.json.get("variable_id", [None])[0]\n\n    if model and experiment and variable:\n        mapping.setdefault(model, {}).setdefault(experiment, set()).add(variable)\n\n# Exibir resultados\nfor model, exps in mapping.items():\n    print(f"\nModelo: {model}")\n    for exp, vars in exps.items():\n        print(f"  Experimento: {exp} → Variáveis: {sorted(vars)}")'

In [57]:
variaveis = ["tas", "tasmin", "tasmax", "tos", "ps", "pr"]  # exemplo com 6 variáveis
modelos_por_variavel = defaultdict(set)

conn = SearchConnection("https://esgf-node.llnl.gov/esg-search", distrib=False)

for var in variaveis:
    ctx = conn.new_context(
        project="CMIP6",
        experiment_id=["historical", "ssp245", "ssp585"],
        variable_id=var,
        frequency="mon",
        replica=False,
        latest=True
    )

    results = list(ctx.search(ignore_facet_check=True))
    for d in results:
        modelo = d.json.get("source_id", [None])[0]
        if modelo:
            modelos_por_variavel[var].add(modelo)

# Exibindo os resultados
for var, modelos in modelos_por_variavel.items():
    print(f"{var}: {sorted(modelos)}")

KeyboardInterrupt: 

In [ ]:
d.json["source_id"]

{'mip_era': ['CMIP6'],
 'activity_drs': ['ScenarioMIP'],
 'institution_id': ['MIROC'],
 'source_id': ['MIROC6'],
 'experiment_id': ['ssp585'],
 'member_id': ['r7i1p1f1'],
 'table_id': ['Amon'],
 'variable_id': ['tas'],
 'grid_label': ['gn'],
 'frequency': ['mon'],
 'realm': ['atmos'],
 'product': ['model-output'],
 'nominal_resolution': ['250 km'],
 'source_type': ['AOGCM', 'AER'],
 'grid': ['native atmosphere T85 Gaussian grid'],
 'creation_date': '2020-06-13T21:31:53Z',
 'variant_label': ['r7i1p1f1'],
 'sub_experiment_id': ['none'],
 'further_info_url': ['https://furtherinfo.es-doc.org/CMIP6.MIROC.MIROC6.ssp585.none.r7i1p1f1'],
 'activity_id': ['ScenarioMIP'],
 'data_specs_version': ['01.00.31'],
 'title': 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp585.r7i1p1f1.Amon.tas.gn',
 'experiment_title': ['update of RCP8.5 based on SSP5'],
 'model_cohort': ['Registered'],
 'data_node': 'esgf-node.ornl.gov',
 'index_node': 'us-index',
 'master_id': 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp585.r7i1p1f1.Amon.